In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import time
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
import speech_recognition as sr
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate', 125)
engine.setProperty('volume',1.0)

    
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    if (msg=="Type a message"):
        if (mic_input.cget("text")!=''):
            msg=mic_input.cget("text")
    EntryBox.delete("0.0",END)

    if msg != 'Type a message':
        ChatLog.config(foreground="#353b48", font=("Segoe Print", 17 ))
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg +'\n')
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        engine.say(res)
        engine.runAndWait()
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        mic_input.config(text="")
    else:
        engine.say(".....OOPS!, Try Again")
        engine.runAndWait()
        
def inp_insert():
    
    EntryBox.delete(0.0,END)
    r=sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source,duration=1)
        engine.say("You can Speak Now")
        engine.runAndWait()
        audio= r.listen(source)
        try:
            text = r.recognize_google(audio)
        except:
            text=".....OOPS! Try Again."
        return text
def speak():
    text=inp_insert()
    if (text==".....OOPS! Try Again."):
        engine.say(text)
        engine.runAndWait()
    mic_input.config(text=text)
    mic_text=mic_input.cget("text")
    EntryBox.focus_force()
    EntryBox.insert(0.0,mic_text)
    
def on_click(event):
    EntryBox.configure(state=NORMAL)
    EntryBox.delete(0.0, END)

    # make the callback only work once
    EntryBox.unbind('<Button-1>', on_click_id)

base = Tk()
base.title("ROSS")
base.geometry("600x700")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bg="#e1b12c", font="Arial",relief='ridge')

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

photo = PhotoImage(file ="mic.png")
photoimage=photo.subsample(3,3)
#Create Button to send message
SendButton = Button(base, width="12", height=5,activebackground="#3c9d9b",fg='#dfe6e9',bg='#1B1464',command=(lambda: send()),text="Send",font=("Verdana",15 ) )

#Create the box to enter message
EntryBox = Text(base, bg="#7f8fa6",width="30", height="15",foreground="#ecf0f1",font=("Verdana", 18 ))
EntryBox.bind("<Return>", (lambda event: send()))

EntryBox.insert(0.0, "Type a message")
EntryBox.configure(state=DISABLED)
on_click_id = EntryBox.bind('<Button-1>', on_click)

MicButton = Button(base, width="12", height=5,activebackground="#3c9d9b",command=speak,text="Record",font=("Verdana",15 ),bg='#f1f1b0',fg='#1B1464' )
mic_input=Label(base,text="",font=("",30),bg='#f1f1b0')

#Place all components on the screen

scrollbar.place(x=576,y=6, height=586)
ChatLog.place(x=6,y=6, height=586, width=565)
EntryBox.place(x=6, y=601, height=90, width=510)
SendButton.place(x=520, y=650,width=75,height=40)
MicButton.place(x=520,y=601,width=75,height=40)
mic_input.place(x=6,y=550,width=565)
base.mainloop()

In [ ]:
import speech_recognition as sr
r=sr.Recognizer()
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source,duration=1)
    print("say anything : ")
    audio= r.listen(source)
    try:
        text = r.recognize_google(audio)
        print(text)
    except:
        print("sorry, could not recognise")

say anything : 


In [38]:
import pyttsx3
engine = pyttsx3.init() # object creation

rate = engine.getProperty('rate')   # getting details of current speaking rate
print (rate)                        #printing current voice rate
engine.setProperty('rate', 125)     # setting up new voice rate


200


In [42]:
voices = engine.getProperty('voices')
print(voices)

[<pyttsx3.voice.Voice object at 0x000001F7C7835108>, <pyttsx3.voice.Voice object at 0x000001F7E226CD08>]


In [39]:
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate', 125)
engine.say("I will speak this text")
engine.runAndWait()

In [33]:
!pip install pyttsx3

In [35]:
!pip install pypiwin32

In [54]:
import pyttsx3 
  
def onStart(): 
    print('starting') 
def onWord(name, location, length): 
    print('word', name, location, length) 
def onEnd(name, completed): 
    print('finishing', name, completed) 
engine = pyttsx3.init() 
engine.setProperty('rate', 100)
engine.connect('started-utterance', onStart) 
engine.connect('started-word', onWord) 
engine.connect('finished-utterance', onEnd) 
  
sen = "You can speak now"
engine.say(sen,'see') 
engine.runAndWait() 

word see 0 3
word see 0 3
word see 0 3
word see 0 3
word see 0 3
word see 0 3
word see 0 3
word see 0 3
word see 0 3
word see 4 3
word see 4 3
word see 4 3
word see 4 3
word see 4 3
word see 4 3
word see 4 3
word see 4 3
word see 4 3
word see 8 5
word see 8 5
word see 8 5
word see 8 5
word see 8 5
word see 8 5
word see 8 5
word see 8 5
word see 8 5
word see 14 3
word see 14 3
word see 14 3
word see 14 3
word see 14 3
word see 14 3
word see 14 3
word see 14 3
word see 14 3
finishing see True
finishing see True
finishing see True
finishing see True
finishing see True
finishing see True
finishing see True
finishing see True
finishing see True
